# Comparison of multiple pipelines and datasets with P300

In this second notebook, we will use P300 paradigm to compare multiple datasets and multiple pipelines. We choose datasets from the BNCI repository.

Please make sure you have verified your installation with the notebook `0_Minischool_Verify_Installation`.

## Selecting datasets

The datasets already defined in MOABB are listed in the [documentation](https://neurotechx.github.io/moabb/api.html). It is also possible to check the [MOABB wiki](https://github.com/NeuroTechX/moabb/wiki/Datasets-Support) for comparative information.

Here we will use two P300 datasets from BNCI:

In [ ]:
from moabb.datasets import BNCI2014008, BNCI2014009

datasets = [BNCI2014008(), BNCI2014009()] 

In [ ]:
from pyriemann.classification import MDM
from pyriemann.tangentspace import TangentSpace
from pyriemann.spatialfilters import Xdawn
from pyriemann.estimation import XdawnCovariances
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC


pipelines = {}
# pipelines['xdawn+svm'] = make_pipeline(Xdawn(nfilter=2), LDA())
pipelines['tgsp+svm'] = make_pipeline(XdawnCovariances(estimator='lwf'), 
                                      TangentSpace(metric='riemann'), 
                                      SVC(kernel='linear'))
pipelines['MDM'] = make_pipeline(XdawnCovariances(estimator='lwf'),
                            MDM(metric='riemann', n_jobs=-1))

In [ ]:
from moabb.paradigms import P300

paradigm = P300()


In [ ]:
from moabb.evaluations import WithinSessionEvaluation

evaluation = WithinSessionEvaluation(paradigm=paradigm, datasets=datasets, overwrite=False)

In [ ]:
results = evaluation.process(pipelines) 

In [ ]:
import seaborn as sns

g = sns.catplot(kind="box", x="score", y="pipeline", aspect=1.5, data=results, orient='h', palette='viridis')

Until now, we have always plotted the results using the seaborn package and creating the figure by ourselves. MOABB also offers some functionalities for analysing the results obtained after running an evaluation procedure.

# Advanced statistical analysis and meta-analysis

For instance, we may create a plot comparing the results with two classification algorithms as in:

In [ ]:
from moabb.analysis.plotting import paired_plot

alg1 = 'tgsp+svm'
alg2 = 'MDM'
fig = paired_plot(results, alg1, alg2)
fig.set_size_inches(9,9)

N.B.: MOABB collapses the values from different sessions into a single average score, which is why we have the impression of having much less points than it should in the plot.

We may also do statistical analysis on the results and plot them with MOABB. For this, we need to first generate an auxiliary dataframe containing all the statistics describing the results and, then, use it as input.

In [ ]:
from moabb.analysis.meta_analysis import compute_dataset_statistics
from moabb.analysis.plotting import meta_analysis_plot

stats_df = compute_dataset_statistics(results)
alg1 = 'tgsp+svm'
alg2 = 'MDM'
fig = meta_analysis_plot(stats_df, alg1, alg2)
fig.set_size_inches(8,7)

In [ ]:
from moabb.analysis.meta_analysis import find_significant_differences
from moabb.analysis.plotting import summary_plot

# stats = compute_dataset_statistics(all_res)
P, T = find_significant_differences(stats_df)
_ = summary_plot(P, T)

# Adapting MOABB to your need

Here are some exercices to introduce some useful possibilities offered by MOABB. 


## Add new datasets in the evaluation (level: easy 🤗)

Use the code above to add another dataset in your evaluation

In [ ]:
datasets = [...]
paradigm = P300()
evaluation = WithinSessionEvaluation(paradigm=paradigm, datasets=datasets, overwrite=True)

results = evaluation.process(pipelines)

## Creating your own pipeline (level: intermediate 🤔)

The first is to create your machine learning pipeline, according to your need. As MOABB use a sklearn-like API for defining pipelines, it is easy to follow the [sklearn tutorial](https://scikit-learn.org/stable/developers/develop.html)

In [ ]:
your_pipelines = {}
your_pipelines['YourPipeline'] = make_pipeline(...)


datasets = [BNCI2014009()]
paradigm = P300()
evaluation = WithinSessionEvaluation(paradigm=paradigm, datasets=datasets, overwrite=True)

results = evaluation.process(your_pipelines)

## Use learning curve (level intermediate 🤔)

Based on the example from the [MOABB gallery](https://neurotechx.github.io/moabb/auto_examples/index.html#evaluation-with-learning-curve) and the [documentation](https://neurotechx.github.io/moabb/generated/moabb.evaluations.WithinSessionEvaluation.html#moabb.evaluations.WithinSessionEvaluation), create an evaluation with an increasing number of sample. For example, using only 1%, 5% and 10% of the data to train. 

In [ ]:
datasets = [...]
paradigm = P300()
evaluation = WithinSessionEvaluation(paradigm=paradigm, datasets=datasets, overwrite=True, ...)

results = evaluation.process(pipelines)